Using -- with MobileNet to make a Monkey Breed Classifier


In [13]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224x224 pixel input images
img_rows, img_cols = 224, 224

# loads the MobileNet model without the top layers or FC layers
MobileNet = MobileNet(weights='imagenet',
                      include_top=False, # put this to True to see ALL layers
                      input_shape=(img_rows, img_cols, 3))

# setting the layers in MobileNet to be non-trainable(True by default)
for layer in MobileNet.layers:
    layer.trainable = False

for (i, layer) in enumerate(MobileNet.layers):
    print(str(i) + '. ' + layer.__class__.__name__ + " - " + str(layer.trainable))

0. InputLayer - False
1. ZeroPadding2D - False
2. Conv2D - False
3. BatchNormalization - False
4. ReLU - False
5. DepthwiseConv2D - False
6. BatchNormalization - False
7. ReLU - False
8. Conv2D - False
9. BatchNormalization - False
10. ReLU - False
11. ZeroPadding2D - False
12. DepthwiseConv2D - False
13. BatchNormalization - False
14. ReLU - False
15. Conv2D - False
16. BatchNormalization - False
17. ReLU - False
18. DepthwiseConv2D - False
19. BatchNormalization - False
20. ReLU - False
21. Conv2D - False
22. BatchNormalization - False
23. ReLU - False
24. ZeroPadding2D - False
25. DepthwiseConv2D - False
26. BatchNormalization - False
27. ReLU - False
28. Conv2D - False
29. BatchNormalization - False
30. ReLU - False
31. DepthwiseConv2D - False
32. BatchNormalization - False
33. ReLU - False
34. Conv2D - False
35. BatchNormalization - False
36. ReLU - False
37. ZeroPadding2D - False
38. DepthwiseConv2D - False
39. BatchNormalization - False
40. ReLU - False
41. Conv2D - False
42. Ba

In [0]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Model
from keras.layers.normalization import BatchNormalization

In [0]:
def addTopModelMobileNet(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(num_classes, activation='softmax')(top_model)

    return top_model

In [16]:
num_classes = 10

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs=MobileNet.input, outputs=FC_Head)

print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [17]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32
train_data_dir = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/monkey-dataset/train'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/monkey-dataset/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical')

Found 1108 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [19]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import RMSprop

save_location = '/content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/saved-models/monkey_breed_mobilenet.h5'

checkpoint = ModelCheckpoint(save_location,
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

callbacks = [earlystop, checkpoint]

# use a very small learning rate
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 1097
nb_validation_samples = 272
epochs = 25
batch_size = 16

history = model.fit_generator( 
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size
)

Epoch 1/25
68/68 [==============================] - 86s 1s/step - loss: 1.4311 - acc: 0.7356 - val_loss: 0.4671 - val_acc: 0.8542

Epoch 00001: val_loss improved from inf to 0.46707, saving model to /content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/saved-models/monkey_breed_mobilenet.h5
Epoch 2/25
68/68 [==============================] - 86s 1s/step - loss: 0.5257 - acc: 0.8526 - val_loss: 0.4169 - val_acc: 0.8457

Epoch 00002: val_loss improved from 0.46707 to 0.41695, saving model to /content/drive/My Drive/Colab Notebooks/Faaltu/CNN/6. Fine Tuning and Transfer Learning/saved-models/monkey_breed_mobilenet.h5
Epoch 3/25
68/68 [==============================] - 87s 1s/step - loss: 0.4387 - acc: 0.8675 - val_loss: 3.3921 - val_acc: 0.5469

Epoch 00003: val_loss did not improve from 0.41695
Epoch 4/25
68/68 [==============================] - 84s 1s/step - loss: 0.5342 - acc: 0.8714 - val_loss: 0.7611 - val_acc: 0.8027

Epoch 00004: val_loss did not i